# Prepare the environment



In [ ]:
%pip install segmentation-models &> /dev/null
%load_ext tensorboard
!wget https://raw.githubusercontent.com/N-Jaro/segmentation_model_tutorial/main/unet_util.py
!cp '/content/drive/MyDrive/Meta_learning_research/Notebooks/data_util.py' .

# https://stackoverflow.com/questions/75433717/module-keras-utils-generic-utils-has-no-attribute-get-custom-objects-when-im
# open the file keras.py, change all the 'init_keras_custom_objects' to 'init_tfkeras_custom_objects'.
# the location of the keras.py is in the error message. In your case, it should be in /usr/local/lib/python3.8/dist-packages/efficientnet/
!wget https://raw.githubusercontent.com/N-Jaro/segmentation_model_tutorial/main/keras.py
!cp './keras.py' '/usr/local/lib/python3.10/dist-packages/efficientnet/keras.py'
!rm './keras.py'

--2023-09-07 18:36:15--  https://raw.githubusercontent.com/N-Jaro/segmentation_model_tutorial/main/unet_util.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12771 (12K) [text/plain]
Saving to: ‘unet_util.py’

unet_util.py        100%[===================>]  12.47K  --.-KB/s    in 0s      

2023-09-07 18:36:16 (76.6 MB/s) - ‘unet_util.py’ saved [12771/12771]

--2023-09-07 18:36:17--  https://raw.githubusercontent.com/N-Jaro/segmentation_model_tutorial/main/keras.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 711 [text/plain]


In [ ]:
# Other dependencies
import random
import sys
import time
import shutil
import numpy as np
import tensorflow as tf
from keras import backend as K
import segmentation_models as sm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from unet_util import dice_coef_loss, dice_coef, jacard_coef, dice_coef_loss, Residual_CNN_block, multiplication, attention_up_and_concatenate, multiplication2, attention_up_and_concatenate2, UNET_224, evaluate_prediction_result
from data_util import *
sm.set_framework('tf.keras')
sm.framework()

# Reproduction
# np.random.seed(333)

print('Python version: ', sys.version)
print('TensorFlow version: ', tf.__version__)


Segmentation Models: using `keras` framework.
Python version:  3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
TensorFlow version:  2.12.0


In [ ]:
%cd '/content/drive/MyDrive/Meta_learning_research/Notebooks/'
import os
input_data = './samples/'
model_path = './models/'
prediction_path = './predicts/'
log_path = './logs/'

# Create the folder if it does not exist
os.makedirs(input_data, exist_ok=True)
os.makedirs(model_path, exist_ok=True)
os.makedirs(prediction_path, exist_ok=True)


/content/drive/MyDrive/Meta_learning_research/Notebooks


In [ ]:
# Avaiable backbones for Unet architechture
# 'vgg16' 'vgg19' 'resnet18' 'resnet34' 'resnet50' 'resnet101' 'resnet152' 'inceptionv3'
# 'inceptionresnetv2' 'densenet121' 'densenet169' 'densenet201' 'seresnet18' 'seresnet34'
# 'seresnet50' 'seresnet101' 'seresnet152', and 'attentionUnet'
backend = 'densenet121' # ResNet50 is the best model in the TL study

# Added first Convo 8 to 3 channels layers to the random init model
name = 'maml-model-' + backend + '-' + str(np.random.randint(1000000))

logdir = log_path + name
if(os.path.isdir(logdir)):
  shutil.rmtree(logdir)
os.makedirs(logdir, exist_ok=True)

print('model location: '+ model_path+name+'.h5')

model location: ./models/maml-model-densenet121-602361.h5


# MAML Loop


In [ ]:
# Create U-net model with the chosen backbone
# The U-net will be initialized with ImageNet weights and the ImageNet weights will be frozen
# in the first pass training

if (backend=="attentionUnet"):
  # Attention U-net model
  learning_rate = 0.0000359
  unet_model = UNET_224()
  unet_model.compile(optimizer = Adam(learning_rate=learning_rate),
                loss = dice_coef_loss,
                metrics = [dice_coef,'accuracy'])
else:
  # Unet with ImageNet backends
  unet_model = sm.Unet(backend, classes = 1, encoder_weights=None, input_shape=(None, None, 8))

  # Compile the model with 'Adam' optimizer (0.001 is the default learning rate) and define the loss and metrics
  unet_model.compile(optimizer = Adam(),
                loss = dice_coef_loss,
                metrics=[dice_coef,'accuracy'])

# Train the unet model
# define hyperparameters and callback modules
patience = 10
maxepoch = 500
callbacks = [ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=patience, min_lr=1e-9, verbose=1, mode='min'),
            EarlyStopping(monitor='val_loss', patience=patience, verbose=0),
            ModelCheckpoint(model_path+name+'.h5', monitor='val_loss', save_best_only=True, verbose=0),
            TensorBoard(log_dir=logdir)]

def f1_score(y_true, y_pred, epsilon=1e-7):
    """
    Compute F1 loss.

    Args:
    - y_true: Tensor of true labels. Shape (batch_size, d0, .. dN).
    - y_pred: Tensor of predicted labels. Shape (batch_size, d0, .. dN).
    - epsilon: Small number to prevent division by zero.

    Returns:
    - f1_loss: F1 loss.
    """
    # Just to make sure the prediction is in the range [0,1]
    y_pred = tf.clip_by_value(y_pred, clip_value_min=0, clip_value_max=1)

    # Compute true positive, false positive, and false negative counts
    tp = tf.reduce_sum(y_true * y_pred, axis=[i for i in range(1, len(y_pred.shape))])
    fp = tf.reduce_sum((1 - y_true) * y_pred, axis=[i for i in range(1, len(y_pred.shape))])
    fn = tf.reduce_sum(y_true * (1 - y_pred), axis=[i for i in range(1, len(y_pred.shape))])

    # Compute precision and recall
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)

    # Compute F1 score
    f1 = 2 * (precision * recall) / (precision + recall + epsilon)

    # Return the mean F1 score across the batch as a loss
    return 1 - tf.reduce_mean(f1)

# MAML inner loop optimizer
optimizer = SGD()

# MAML outer loop optimizer
outer_optimizer = Adam()

# save the training history
train_history_all = []



# MAML outer loop function
def outer_loop_fn(episodes, num_tasks):
    with tf.GradientTape(watch_accessed_variables=False) as tape:
      losses = []
      tape.watch(losses)

      # Compute loss on multiple tasks
      losses = [inner_loop_fn(episodes[i]) for i in range(num_tasks)]

      # Compute mean loss across tasks
      mean_loss = tf.reduce_mean(losses)

      # Compute gradients for outer loop
      grads = tape.gradient(mean_loss, unet_model.trainable_variables)

      # Apply gradients to U-net model
      outer_optimizer.apply_gradients(zip(grads, unet_model.trainable_variables))


In [ ]:
def inner_loop_fn(episode):
    with tf.GradientTape() as tape:
        # Forward pass through U-net model on support set
        support_logits = unet_model(episode['support_set_data'][:5].astype(np.float32))

        # Compute F1-score loss for support set
        support_loss = f1_score(episode['support_set_labels'][:5].astype(np.float32), support_logits)

    # Compute gradients for support set loss
    grads = tape.gradient(support_loss, unet_model.trainable_variables)

    # Apply gradients to U-net model
    optimizer.apply_gradients(zip(grads, unet_model.trainable_variables))

    # Now compute the loss on the query set after updating the model with support set gradients
    query_logits = unet_model(episode['query_set_data'][:5].astype(np.float32))
    query_loss = f1_score(episode['query_set_labels'][:5].astype(np.float32), query_logits)

    return query_loss

In [ ]:
# MAML inner loop function
def inner_loop_fn(episode):

    def closure():
        with tf.GradientTape() as tape:
            # Forward pass through U-net model
            logits = unet_model(episode['query_set_data'][:5].astype(np.float32))

            # Compute F1-score loss
            loss = f1_score(episode['query_set_labels'][:5].astype(np.float32), logits)

        # Compute gradients
        grads = tape.gradient(loss, unet_model.trainable_variables)

        # Apply gradients to U-net model
        optimizer.apply_gradients(zip(grads, unet_model.trainable_variables))

        print(loss)

        return loss

    optimizer.minimize(closure, var_list=unet_model.trainable_variables)

In [ ]:
def outer_loop_fn(episodes, num_tasks):
    accumulated_grads = [tf.zeros_like(var) for var in unet_model.trainable_variables]

    train_history = unet_model.fit(x = episode['support_set_data'].astype(np.float32),y = episode['support_set_labels'].astype(np.float32),
                              validation_data = (episode['query_set_data'].astype(np.float32), episode['query_set_labels'].astype(np.float32)),
                              batch_size = 16, epochs = maxepoch, verbose=1, callbacks = callbacks)

    train_history_all.append(train_history)

    for i in range(num_tasks):
        with tf.GradientTape(watch_accessed_variables=False) as tape:
            tape.watch(unet_model.trainable_variables)

            loss = inner_loop_fn(episodes[i])

            grads = tape.gradient(loss, unet_model.trainable_variables)
            accumulated_grads = [acc_grad + grad for acc_grad, grad in zip(accumulated_grads, grads)]

    # Average the gradients over the number of tasks
    averaged_grads = [grad / num_tasks for grad in accumulated_grads]

    # Apply gradients
    outer_optimizer.apply_gradients(zip(averaged_grads, unet_model.trainable_variables))


## Prepare data episodes

In [ ]:
%cd '/content/drive/MyDrive/Meta_learning_research/Notebooks/'
data_dir = './samples/'  # Replace with the path to your directory containing numpy files
locations_meta_training = ['Alexander', 'Rowancreek']
locations_meta_testing = ['Covington']
num_samples_per_location = 100  # Configure the number of samples per location
num_episodes = 10  # Number of episodes

data_loader = MetaDataLoader(data_dir, num_samples_per_location)

# Create multi episodes for meta-training
mate_train_episodes = data_loader.create_multi_episodes(num_episodes, locations_meta_training)
mate_test_episodes = data_loader.create_multi_episodes(num_episodes, locations_meta_testing)

/content/drive/MyDrive/Meta_learning_research/Notebooks


## Training Loop

In [ ]:
num_epochs = 10  # Configure the number of epochs

# Training loop
for i in range(num_epochs):
    outer_loop_fn(mate_train_episodes, 2)
    print('Epoch: ' + str(i))

ResourceExhaustedError: ignored

In [ ]:
# Use the trained U-net model to segment input image
segmented_image = unet_model(mate_test_episodes[0]['query_set_data'])

np.save(prediction_path+name+'_predict.npy',segmented_image)

In [ ]:
from google.colab import runtime
runtime.unassign()